# Newton Polynomial

Given a distinct (no two $x_{j}$ are the same) set of $n + 1$ data points
\begin{align*}
(x_{0},y_{0}),\ldots ,(x_{j},y_{j}),\ldots ,(x_{n},y_{n}).
\end{align*}
For these sets of data points, we can define the forward divided differences as follows:

\begin{align*}
f[x_{k }] &=y_{k },\qquad k \in \{0,\ldots ,n\},\\
f[x_{k },~x_{k +1}]&= \frac  {y_{k+1 } - y_{k }}{x_{{k +1}}-x_{k }},\qquad k \in \{0,\ldots ,n-1\},\\
f[x_{k },~x_{k +1},~x_{k +2}]&= \frac  {f[x_{k+1 },~x_{k +2}] - f[x_{k },x_{k +1}]}{x_{{k +2}}-x_{k }},\qquad k \in \{0,\ldots ,n-2\},\\
f[x_{k },~x_{k +1},~x_{k +2},~x_{k +3}]&= \frac  {f[x_{k+1 },~x_{k +2},~x_{k +3}] - f[x_{k },~x_{k +1},~x_{k +2}]}{x_{{k +3}}-x_{k }},\qquad k \in \{0,\ldots ,n-3\},\\
 &~~ \vdots\\
f[x_{k },\ldots ,~x_{{k +j}}]&={\frac  {f[x_{{k +1}},\ldots ,~x_{{k +j}}]-f[x_{{k }},\ldots ,~x_{{k +j-1}}]}{x_{{k +j}}-x_{k }}},\qquad k \in \{0,\ldots ,n-j\},\ j\in \{1,\ldots ,n\}.
\end{align*}
Now, let
\begin{align*}
a_{j} &= f[x_{0},~x_{1},\ldots ,~x_{{j}}], \qquad j\in \{1,\ldots ,n\},\\
n_{j}(x)&=
\begin{cases}
1, & j = 0,\\
\prod _{{i=0}}^{{j-1}}(x-x_{i}), & j>0.
\end{cases}
\end{align*}
to be Newton coefficients and Newton basis polynomials, respectively. Then, the Newton interpolation polynomial is a linear combination of Newton basis polynomials
\begin{align*}
N(x):=\sum _{{j=0}}^{{k}}a_{{j}}n_{{j}}(x).
\end{align*}

Furthermore, we can prepare a Python code using the above algorithm.

In [1]:
import numpy as np
import pandas as pd
def NewtonCoeff(xn, yn, table = False):
    '''
    Input: data points
    Output: Newton coefficients
    '''
    n = len(xn)
    #Construct table and load xy pairs in first columns
    A = np.zeros((n,n+1))
    A[:,0]= xn[:]
    A[:,1]= yn[:]
    #Fill in Divided differences
    for j in range(2,n+1):
        for i in range(j-1,n):
            A[i,j] = (A[i,j-1]-A[i-1,j-1]) / (A[i,0]-A[i-j+1,0])
    #Copy diagonal elements into array for returning
    a = np.zeros(n)
    for k in range(0,n):
        a[k] = A[k,k+1]
    if table:
        Cols = ['xi','yi']
        temp = 'f[xi'
        for i in range(A.shape[1]-2):
            temp += ', xi+%i'% (i+1)
            Cols.append(temp + ']')
        Table = pd.DataFrame(A, columns=Cols)
        return a, Table
    else:
        return a

def NewtonPoly(x, xn, yn):
    # xn: data points at x
    # yn: data points at y
    # x: evaluation point(s)
    
    n = len(xn)
    # coefficients
    a = NewtonCoeff(xn, yn)
    p = a[n-1]
    for i in range(n-2,-1,-1):
        p = p*(x-xn[i]) + a[i]
    return p

<font color='Blue'><b>Example</b></font>: Consider the following data points
$$\{(1,-3),~(2,0),~(3,-1),~(4,2),~(5,1),~(6,4)\}$$
and apply the Newton Method.

In [2]:
# This part is used for producing tables and figures
import sys
sys.path.insert(0,'..')
import hd_tools as hd

Loading BokehJS ...

In [3]:
# A set of distinct points
xn = np.array ([-2, 1 , 3 , 5 , 6, 7])
yn = np.array ([-5, -3 ,-1 , 1 , 4, 10])

_, Table = NewtonCoeff(xn, yn, table = True)
display(Table.style.set_properties(subset=['xi', 'yi'], **{'background-color': 'black', 'color': 'PaleGreen',
       'border-color': 'DarkGreen'}).format({Table.columns.tolist()[-1]: "{:.4e}"}))
x = np.linspace( xn.min()-1 , xn.max()+1 ,100)
y = NewtonPoly( x , xn , yn )
hd.interpolation_method_plot(xn, yn, x, y, title = 'Newton Method')

,xi,yi,"f[xi, xi+1]","f[xi, xi+1, xi+2]","f[xi, xi+1, xi+2, xi+3]","f[xi, xi+1, xi+2, xi+3, xi+4]","f[xi, xi+1, xi+2, xi+3, xi+4, xi+5]"
0,-2.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.0000e+00
1,1.000000,-3.000000,0.666667,0.000000,0.000000,0.000000,0.0000e+00
2,3.000000,-1.000000,1.000000,0.066667,0.000000,0.000000,0.0000e+00
3,5.000000,1.000000,1.000000,0.000000,-0.009524,0.000000,0.0000e+00
4,6.000000,4.000000,3.000000,0.666667,0.133333,0.017857,0.0000e+00
5,7.000000,10.000000,6.000000,1.500000,0.208333,0.012500,-5.9524e-04


## Newton Forward Differences

Given equidistantly distributed data points, we can define Newton forward differences. Note that this is a special case of Newton polynomials. We have,

Given n data points

\begin{align*}
\left\{(x_{0},y_{0}),~(x_{1},y_{1}),\ldots ,(x_{n},y_{n}) \right\}
\end{align*}
where
\begin{align*}
x_{j} = x_{0} + jh,\qquad h>0.
\end{align*}
Note that $h = \Delta x_{j} = x_{j+1} -x_{j}$

Then, the divided differences can be calculated via forward differences defined as
\begin{align*}
f[x_{0}] &=y_{0 },\\
f[x_{0 },~x_{1}]&= \frac  {y_{1 } - y_{0 }}{x_{{1}}-x_{k }} = \frac{\Delta y_{0 }}{h},\\
f[x_{0 },~x_{1},~x_{2}]&= \frac  {f[x_{1 },~x_{2}] - f[x_{1 },x_{0}]}{x_{2}-x_{0 }}
= \frac  {1}{2h}\left(\frac{\Delta y_{1}}{h} - \frac{\Delta  y_{0}}{h}\right)
= \frac{1}{2!h^2}\Delta^2 y_{0 },\\
 &~~ \vdots\\
f[x_{0},\ldots ,~x_{j}]&=\frac{1}{j!h^j}\Delta^j y_{0 }
,\qquad j\in \{1,\ldots ,n\}.
\end{align*}
Therefore, the Newton Forward-Difference Formula can be found as follows,
\begin{align*}
P_{n}(x) = y_{0} + \sum_{k = 1}^{n} \binom{s}{k} \Delta^{k} y_{0},
\end{align*}
where $s = \dfrac{x - x_{0}}{h}$. Furthermore, we can prepare a Python code using the above algorithm.

In [4]:
from scipy.special import binom
def ForwardNewton(xn, yn, table = False):
    '''
    Input: data points
    Output: Newton coefficients
    '''
    h = xn[1]- xn[0]
    n = xn.shape[0]
    A = np.zeros([n,n],dtype = float)
    A[:,0] = yn
    for i in range(1, n):
        A[i:,i] = (A[i:,i-1] - A[i-1:-1,i-1])
    df = np.diag(A)
    # Newton's forward difference formula for variable x
    def Pn(x, df = df, h = h, xn = xn):
        s = (x - xn[0])/h
        P = df[0]
        for i in range(1, len(df)):
            P += binom(s, i)*df[i]
        return P
    if table:
        Cols = ['xi','yi']
        for i in range(A.shape[1]-1):
            Cols.append('Delta^%i fi'% (i+1))
        Table = pd.DataFrame(np.insert(A, 0, xn, axis=1), columns=Cols)
        return Pn, Table
    else:
        return Pn

In [5]:
xn = np.array ([1 ,2 ,3 ,4 ,5 , 6])
yn = np.array ([-3 ,0 ,-1 ,2 ,1 , 4])

Pn, Table = ForwardNewton(xn, yn, table = True)
display(Table.style.set_properties(subset=['xi', 'yi'], **{'background-color': 'black', 'color': 'PaleGreen',
       'border-color': 'DarkGreen'}).format(precision=4))
x = np.linspace(xn.min()-1 , xn.max()+1 , 100)
y = Pn(x)
hd.interpolation_method_plot(xn, yn, x, y, title = 'Forward Newton Method')

,xi,yi,Delta^1 fi,Delta^2 fi,Delta^3 fi,Delta^4 fi,Delta^5 fi
0,1.0000,-3.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,2.0000,0.0000,3.0000,0.0000,0.0000,0.0000,0.0000
2,3.0000,-1.0000,-1.0000,-4.0000,0.0000,0.0000,0.0000
3,4.0000,2.0000,3.0000,4.0000,8.0000,0.0000,0.0000
4,5.0000,1.0000,-1.0000,-4.0000,-8.0000,-16.0000,0.0000
5,6.0000,4.0000,3.0000,4.0000,8.0000,16.0000,32.0000


## Newton Backward Differences

Similarly, here, given n data points

\begin{align*}
\left\{(x_{0},y_{0}),~(x_{1},y_{1}),\ldots ,(x_{n},y_{n}) \right\}
\end{align*}
where
\begin{align*}
x_{j} = x_{0} + jh,\qquad h>0.
\end{align*}
Note that $h = \Delta x_{j} = x_{j+1} -x_{j}$

Then, the divided differences can be calculated via forward differences defined as
\begin{align*}
f[x_{n}] &=y_{n },\\
f[x_{n },~x_{n-1}]&= \frac  {y_{n} - y_{n-1 }}{x_{{1}}-x_{k }} = \frac{\nabla y_{n }}{h},\\
f[x_{n },~x_{n-1},~x_{n-2}]&= \frac  {f[x_{n },~x_{n-1}] - f[x_{n-1 },x_{n-2}]}{x_{n}-x_{n-2 }}
= \frac  {1}{2h}\left(\frac{\nabla y_{n}}{h} - \frac{\nabla  y_{n-1}}{h}\right)
= \frac{1}{2!h^2}\nabla^2 y_{n },\\
 &~~ \vdots\\
f[x_{0},\ldots ,~x_{n-j}]&=\frac{1}{j!h^j}\nabla^j y_{n }
,\qquad j\in \{0,\ldots ,n-1\}.
\end{align*}
Therefore, the Newton Backward-Difference Formula can be found as follows,
\begin{align*}
P_{n}(x) = y_{0} + \sum_{k = 1}^{n} \left(-1\right)^k\binom{-s}{k} \nabla^{k} y_{n},
\end{align*}
where $s = \dfrac{x - x_{0}}{h}$. Furthermore, we can prepare a Python code using the above algorithm.

In [6]:
def BackwardNewton(xn, yn, table = False):
    '''
    Input: data points
    Output: Newton coefficients
    '''
    h = xn[1]- xn[0]
    n = xn.shape[0]
    A = np.zeros([n,n],dtype = float)
    A[:,0] = yn
    B = A.copy()
    for i in range(1, n):
        A[i:,i] = (A[i:,i-1] - A[i-1:-1,i-1])
        B[:-i,i] = (A[i:,i-1] - A[i-1:-1,i-1])
    df = A[-1,:]
    A = B.copy()
    del B
    # Newton's forward difference formula for variable x
    def Pn(x0, df = df, h = h, xn = xn):
        s = (x0 - xn[-1])/h
        P = df[0]
        for i in range(1, len(df)):
            if df[i]!=0:
                T = s
                for j in range(1, i):
                    T *= (s+j)
                T /= np.math.factorial(i)
                P += T*df[i]
                del T
        return P
    if table:
        Cols = ['xi','yi']
        for i in range(A.shape[1]-1):
            Cols.append('Delta^%i fi'% (i+1))
        Table = pd.DataFrame(np.insert(A, 0, xn, axis=1), columns=Cols)
        return Pn, Table
    else:
        return Pn

In [7]:
xn = np.array ([1 ,2 ,3 ,4 ,5 , 6])
yn = np.array ([-3 ,0 ,-1 ,2 ,1 , 4])

Pn, Table = BackwardNewton(xn, yn, table = True)
display(Table.style.set_properties(subset=['xi', 'yi'], **{'background-color': 'black', 'color': 'PaleGreen',
       'border-color': 'DarkGreen'}).format(precision=4))
x = np.linspace(xn.min()-1 , xn.max()+1 , 100)
y = np.array ([Pn(x[i]) for i in range(len(x))])
    
hd.interpolation_method_plot(xn, yn, x, y, title = 'Backward Newton Method')

,xi,yi,Delta^1 fi,Delta^2 fi,Delta^3 fi,Delta^4 fi,Delta^5 fi
0,1.0000,-3.0000,3.0000,-4.0000,8.0000,-16.0000,32.0000
1,2.0000,0.0000,-1.0000,4.0000,-8.0000,16.0000,0.0000
2,3.0000,-1.0000,3.0000,-4.0000,8.0000,0.0000,0.0000
3,4.0000,2.0000,-1.0000,4.0000,0.0000,0.0000,0.0000
4,5.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000
5,6.0000,4.0000,0.0000,0.0000,0.0000,0.0000,0.0000


***
**References:**
1. Allaire, Grégoire, et al. Numerical linear algebra. Vol. 55. New York: Springer, 2008.
1. Burden, Richard L., and J. Douglas Faires. "Numerical analysis 8th ed." Thomson Brooks/Cole (2005).
1. Atkinson, Kendall E. An introduction to numerical analysis. John wiley & sons, 2008.
1. Khoury, Richard, and Douglas Wilhelm Harder. Numerical methods and modelling for engineering. Springer, 2016.
1. Zarowski, Christopher J. An introduction to numerical analysis for electrical and computer engineers. John Wiley & Sons, 2004.
1. [Newton polynomial Wikipedia page](https://en.wikipedia.org/wiki/Newton_polynomial)
1. [Divided differences Wikipedia page](https://en.wikipedia.org/wiki/Divided_differences)
***